# Multiple Linear Regression: Boston House Prediction

* CRIM per capita crime rate by town
* ZN proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS proportion of non-retail business acres per town
* CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
* NOX nitric oxides concentration (parts per 10 million)
* RM average number of rooms per dwelling
* AGE proportion of owner-occupied units built prior to 1940
* DIS weighted distances to five Boston employment centres
* RAD index of accessibility to radial highways
* TAX full-value property-tax rate per 10,000usd
* PTRATIO pupil-teacher ratio by town
* B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT % lower status of the population

In [426]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import f as fdist
from scipy.stats import t as tdist
from scipy.stats import chi2 as chisq

In [305]:
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
         'DIS', 'RAD', 'TAX', 'PTRATIO','B', 'LSTAT', 'MEDV']
df = pd.read_csv('data/housing.data', header=None, delim_whitespace=True, names=names)

In [306]:
df = df.select_dtypes(float)

In [307]:
df.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,222.0,18.7,396.90,5.33,36.2


In [308]:
corr = abs(df.corr()['MEDV']).sort_values(ascending=False)
clmns = corr[1:3].index

In [309]:
df = df.loc[:, list(clmns) + ['MEDV']]
df.head()

,LSTAT,RM,MEDV
0,4.98,6.575,24.0
1,9.14,6.421,21.6
2,4.03,7.185,34.7
3,2.94,6.998,33.4
4,5.33,7.147,36.2


## Create the regressor X matrix and regressand y matrix

$ y = \beta_{1} + \beta_{2}X_{2} + \beta_{3}X_3 + u_{i} $ <br/>

In [310]:
X = df.iloc[:, 0:2].values
X = np.hstack((np.ones((X.shape[0], 1)), X))

y = df.iloc[:, -1].values.reshape(-1, 1)

In [311]:
X

array([[1.   , 4.98 , 6.575],
       [1.   , 9.14 , 6.421],
       [1.   , 4.03 , 7.185],
       ...,
       [1.   , 5.64 , 6.976],
       [1.   , 6.48 , 6.794],
       [1.   , 7.88 , 6.03 ]])

In [312]:
X.shape, y.shape

((506, 3), (506, 1))

In [313]:
n, k = X.shape


## Find the estimate of Beta

In [314]:
import numpy as np
from numpy.linalg import inv

In [315]:
A = inv(X.T@X)

In [316]:
beta_hat = A@X.T@y
beta_hat

array([[-1.35827281],
       [-0.64235833],
       [ 5.09478798]])

## Obtain the estimate of y (predicted y)

In [317]:
y_hat = X@beta_hat

## Compute the residual sum of squares

In [318]:
u_hat = y - y_hat
rss = (u_hat.T@ u_hat).ravel()[0]

## Obtain the variance of beta_hat

In [319]:
beta_hat_var = (rss/(n-k))*(A)

In [320]:
beta_hat_var

array([[ 1.00668361e+01, -9.91781334e-02, -1.39248641e+00],
       [-9.91781334e-02,  1.91244101e-03,  1.19306696e-02],
       [-1.39248641e+00,  1.19306696e-02,  1.97549581e-01]])

## Create the regression table

## Create the table for hypothesis testing

In [321]:
from scipy.stats import t as tdist

In [322]:
beta_hat_stderr = np.diag(beta_hat_var)**(1/2)

In [323]:
beta_hat.flatten()

array([-1.35827281, -0.64235833,  5.09478798])

In [324]:
[beta_hat.flatten(), beta_hat_stderr]

[array([-1.35827281, -0.64235833,  5.09478798]),
 array([3.17282778, 0.04373146, 0.4444655 ])]

In [325]:
df_hyp = pd.DataFrame({'Estimate': beta_hat.flatten(),
                       'Standard_error': beta_hat_stderr})

df_hyp['t_statistic'] = df_hyp['Estimate']/df_hyp['Standard_error']

In [326]:
df_hyp['pval'] = tdist.cdf(-abs(df_hyp.t_statistic), n-k)*2

In [327]:
from scipy.stats import t as tdist

critical_val = abs(tdist.ppf(q=[0.025], df=n-k))[0]
critical_val

1.9646914053628335

In [328]:
df_hyp['significant_0.05'] = (abs(df_hyp['t_statistic']) > critical_val)
df_hyp

,Estimate,Standard_error,t_statistic,pval,significant_0.05
0,-1.358273,3.172828,-0.428095,6.687649e-01,False
1,-0.642358,0.043731,-14.688699,6.669365e-41,True
2,5.094788,0.444466,11.462730,3.472258e-27,True


## Calculate TSS, ESS and RSS

In [329]:
tss = y.var()*y.size
ess = tss - rss

In [330]:
R_sq = (ess/tss)
R_sq_adj = (1 - (rss/(n-k))/(tss/(n-1)))

print('R squared: {:.4f}'.format(R_sq))
print('R squared (Adjusted): {:.4f}'.format(R_sq_adj))

R squared: 0.6386
R squared (Adjusted): 0.6371


# 

## Make the LinearRegression Model

In [331]:
from sklearn.base import BaseEstimator, ClassifierMixin

In [485]:
class LinearReg(BaseEstimator, ClassifierMixin):
    
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
    
    @staticmethod
    def _remake_X(X):
        if not np.all(X == 0):
            return np.hstack((np.ones((X.shape[0], 1)), X))
        else:
            return np.ones((X.shape[0], 1))

    @staticmethod
    def _remake_y(y):
        return y.reshape(-1, 1) # Check before reshaping?

    
    def fit(self, X, y):
        
        # Make proper X and y matrix
        if self.fit_intercept:
            if X is not None:
                X = self._remake_X(X)
            else:
                X = np.zeros((y.shape[0]))
                X = np.hstack((np.ones((y.shape[0], 1)), X))
            
        y = self._remake_y(y)
        
        # Store n and k
        n, k = X.shape
        
        # Obtain estimate of beta =  beta_hat
        try:
            A = inv(X.T@X)
        except np.linalg.LinAlgError:
            A = np.zeros([X.shape[1]]*2)
            
        beta_hat = A@X.T@y
        
        # Obtain estimate of y = y_hat
        y_hat = X@beta_hat
        
        # Compute residual sum of squares
        u_hat = y - y_hat
        rss = (u_hat.T @ u_hat).ravel()[0]
        
        # Estimate the variance of beta_hat
        beta_hat_var = (rss/(n-k))*A
        
        # Calculate tss, ess and rss
        tss = y.var()*n
        ess = tss - rss
        
        # Calculate R-squared and Adjusted R-squared
        rsq = ess/tss
        rsq_adj = 1 - (rss/(n-k))/(tss/(n-1))
        
        # Perform F-test for overall significance
        dfn, dfd = k-1, n-k
        fstat = (ess/(dfn))/(rss/(dfd))
        ftest_pval =  1 - fdist.cdf(fstat, dfn, dfd)
        
        # store
        self.n = n
        self.k = k
        self.beta_hat = beta_hat
        self.rss = rss
        self.beta_hat_var = beta_hat_var
        self.u_hat = u_hat
        self.rsq = rsq
        self.rsq_adj = rsq_adj
        self.aic = rss/n + 2*k/n
        self.bic = rss/n + 2*k*np.log(n)/n
        self.fstat = fstat
        self.ftest_pval = ftest_pval
    
        return self
    
        
    def predict(self, X):
        X = self._remake_X(X)
        return X@self.beta_hat
    
    def predict_interval(self, X):
        raise NotImplementedError('This feature is not implemented yet!')
        
    
    def score(self, X, y):
        lm = LinearReg(self.fit_intercept)
        lm.fit(X, y)
        return lm.rsq_adj
    

In [486]:
X = df.iloc[:, 0:2].values
y = df.iloc[:, -1].values.reshape(-1, 1)

In [487]:
lm = LinearReg()
lm.fit(X, y)

LinearReg(fit_intercept=True)

In [488]:
lm.score(X, y)

0.6371244754701231

In [489]:
from sklearn.model_selection import cross_val_score

In [490]:
-cross_val_score(lm, X, y, scoring="neg_mean_squared_error")

array([11.78879381, 28.97215949, 47.84395882, 71.77053822, 36.6099291 ])

In [491]:
from sklearn.linear_model import LinearRegression
lm_sk = LinearRegression()

-cross_val_score(lm_sk, X, y, scoring="neg_mean_squared_error")

array([11.78879381, 28.97215949, 47.84395882, 71.77053822, 36.6099291 ])

In [492]:
cross_val_score(lm_sk, X, y, cv=10) # Which default scoring metric does this use?

array([ 0.66158301,  0.64905012, -1.51946963,  0.49331255,  0.59340312,
        0.47386502, -0.17695506,  0.05396455, -1.77172578,  0.21261084])

## Joint Hypothesis

$ H_{0}: \beta_{2} +\beta_{3} = 0$ <br/>
$ H_{1}: \beta_{2} +\beta_{3} \neq 0$

In [493]:
lm = LinearReg()

In [494]:
# Unrestricted residual sum of squares
urss = lm.fit(X, y).rss

In [495]:
X.shape

(506, 2)

In [496]:
X_star = (X[:, 0] - X[:, 1]).reshape(-1, 1)
y_star = y - X[:, 1].reshape(-1, 1)

In [497]:
# Restricted residual sum of squares
rrss = lm.fit(X_star, y_star).rss

num_restr = X_star.shape[1]
n = X.shape[0]
k = X.shape[1]
F_stat = ((rrss - urss)/num_restr)/(urss/(n-k))

print('F statistic: {:.5f}'.format(F_stat))
print('Critical value: {:.5f}'.format(fdist.ppf(1 - 0.05, num_restr, n-k)))

F statistic: 53.47836
Critical value: 3.85998


In [498]:
class IncompatibleLinearModelError(Exception):
    pass

def Ftest_LinearReg(lm1: LinearReg, lm2: LinearReg, sig):
    '''Perform the Ftest for two linear model fits of class LinearReg
    
    The restricted and unrestricted models are automatically assigned
    by accessing the k attribute of lm1 and lm2.
    
    Parameters
    ----------
    
    lm1, lm2: LinearReg
        fitted models of class LinearReg
        
    sig: int, optional (default=0.05)
        the significance level of the test
        
    Return -> True if the test is significant
    
    '''
    if (lm1.n == lm2.n) and (lm1.k != lm2.k): # NOTE: Python 3.8 assign n to a var 
        n = lm1.n
    else:
        raise(IncompatibleLinearModelError(
            'The linear models were fitted on datasets with varying observations')
             )
    
    lms = [lm1, lm2]
    r = np.argmin([lm.k for lm in lms])
    ur = [i for i in range(len(lms)) if i != r][0]
    
    lm_r, lm_ur = lms[r], lms[ur]
    
    rrss, urss = lm_r.rss, lm_ur.rss
    dfn = lm_ur.k - lm_r.k
    dfd = n - lm_ur.k
    
    F_stat = ((rrss - urss)/dfn)/(urss/dfd)
    critical_val = fdist.ppf(1 - sig, dfn, dfd)
    pval = 1 - fdist.cdf(F_stat, dfn, dfd)
    
    print('F statistic: {:.4f} with dfn: {}, and dfd: {}'.format(F_stat, dfn, dfd))
    print('Critical value: {:.4f}'.format(critical_val))
    print('p-value: {:.4f}'.format(pval))
    
    if F_stat > critical_val:
        return True
    
    return False

In [499]:
lm_r = LinearReg().fit(X, y)
lm_ur = LinearReg().fit(X_star, y_star)
Ftest_LinearReg(lm_r, lm_ur, 0.05)

F statistic: 53.3723 with dfn: 1, and dfd: 503
Critical value: 3.8600
p-value: 0.0000


True

Now let's do something that might make us **NOT** reject the null hypothesis.

In [500]:
lm.fit(X, y)
beta_hat = lm.beta_hat
beta_hat[1] + beta_hat[2]

array([4.45242965])

$ H_{0}: \beta_{2} +\beta_{3} = 4.47$ <br/>
$ H_{1}: \beta_{2} +\beta_{3} \neq 4.47$

In [501]:
X_star = (X[:, 0] - X[:, 1]).reshape(-1, 1)
y_star = y - 4.47*X[:, 1].reshape(-1, 1)

In [502]:
lm_ur = LinearReg().fit(X, y)
lm_r = LinearReg().fit(X_star, y_star)

In [503]:
Ftest_LinearReg(lm_ur, lm_r, 0.05)

F statistic: 0.0014 with dfn: 1, and dfd: 503
Critical value: 3.8600
p-value: 0.9704


False

Let's check if the pvalue from the F-test matches with the t-test for the intercept case <br/>
$ H_{0}: \beta_{1} = 0$ <br/>
$ H_{1}: \beta_{1} \neq 0$

In [504]:
lm_r = LinearReg(fit_intercept=False).fit(X, y)
lm_ur = LinearReg(fit_intercept=True).fit(X, y)

Ftest_LinearReg(lm_r, lm_ur, 0.05)

F statistic: 0.1833 with dfn: 1, and dfd: 503
Critical value: 3.8600
p-value: 0.6688


False

And Voila! it matches :D

In [505]:
df_hyp

,Estimate,Standard_error,t_statistic,pval,significant_0.05
0,-1.358273,3.172828,-0.428095,6.687649e-01,False
1,-0.642358,0.043731,-14.688699,6.669365e-41,True
2,5.094788,0.444466,11.462730,3.472258e-27,True


## Test for Heteroskedasticity

### Breush-Pagan test

$H_{0}: \alpha_{2} = \alpha_{3} = 0$ <br />
$H_{1}: atleast \ one \ \alpha_{i} \neq 0  \ \forall \ i = 1, 2, ...k$

In [506]:
lm = LinearReg().fit(X, y)

# residual
res = y - lm.predict(X)
assert((res**2).sum() == lm.rss)

# residual squared
res_sq = res**2

# Null Hypothesis
X_H0 = None
lm_H0 = LinearReg().fit(None, res_sq)

# Alternate Hypothesis
# Now regress the res_sq on X
lm_H1 = LinearReg().fit(X, res_sq)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [452]:
Ftest_LinearReg(lm_H0, lm_H1, 0.05)

F statistic: 0.76264 with dfn: 2, and dfd: 503
Critical value: 3.01365
p-value: 0.46697


False

In [481]:
def breush_pagan(X: np.array, y: np.array, sig):
    ''' Breush Pagan test for heteroskedasticity
    
    Parameters
    ----------
    X: np.array
        Regressor matrix
    
    y: np.array
        Regressand matrix
        
    '''
    lm = LinearReg().fit(X, y)

    # residual
    res = y - lm.predict(X)

    # residual squared
    res_sq = res**2

    # Null Hypothesis
    lm_H0 = LinearReg().fit(None, res_sq)

    # Alternate Hypothesis
    # Now regress the res_sq on X
    lm_H1 = LinearReg().fit(X, res_sq)
    
    return(Ftest_LinearReg(lm_H0, lm_H1, sig))

In [482]:
# Refer: t.ly/gr9v6 This one uses a chisq statistic
def breusch_pagan2(X, y, sig):
    ''' Breush Pagan test for heteroskedasticity
    
    Parameters
    ----------
    X: np.array
        Regressor matrix
    
    y: np.array
        Regressand matrix
        
    '''
    
    lm = LinearReg().fit(X, y)

    # residual
    res = y - lm.predict(X)

    # residual squared
    res_sq = res**2

    # Alternate Hypothesis
    lm_H1 = LinearReg().fit(X, res_sq)
    k = lm_H1.k
    rsq = lm_H1.rsq
    n = lm_H1.n
    
    # n*Rsq is chisq distributed with k-1 df
    chisq_stat = n*rsq
    df = k-1
    pval = 1 - chisq.cdf(chisq_stat, df)
    critical_val = chisq.ppf(1- sig, df)
    
    print('Chisq statistic: {:.4f} with df: {}'.format(chisq_stat, k-1))
    print('Critical value: {:4f}'.format(critical_val))
    print('pval: {:4f}'.format(pval))
    
    if chisq_stat > critical_val:
        return True
    
    return(False)

In [483]:
breusch_pagan2(X, y, 0.05)

Chisq statistic: 1.5297 with df: 2
Critical value: 5.991465
pval: 0.465394


False

## Rectifying Heteroskedasticity

Doubts: <br/>
1. 

In [360]:
z_hat = lm_H1.predict(X)

In [477]:
X_star = LinearReg._remake_X(X)/(z_hat)**(1/2)
y_star = y/(z_hat)**(1/2)
lm_hk = LinearReg(fit_intercept=False).fit(X_star, y_star)

In [478]:
# Significantly reduced rss but this may be overfit
lm_hk.rss

527.7243770112723

In [479]:
lm = LinearReg().fit(X, y)
lm.rss

15439.30920131353